In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()


In [ ]:
def list_diseases():
    query = "SELECT DISTINCT trait_reported FROM `thegwensproject.genomiX.disease_variant_gene` order by trait_reported;"
    query_job = client.query(query)  # Make an API request.
    return query_job.to_dataframe()

In [ ]:
def count_variants(chr, pos):
    if pos == 0:
        query = (
            "SELECT chr_id_b37, position_b37, COUNT(*) as count FROM thegwensproject.genomiX.variants WHERE chr_id_b37 = '" +str(chr) +"' GROUP BY chr_id_b37, position_b37 HAVING COUNT(*) > 1 LIMIT 20;")
    else:
        query = (
            "SELECT chr_id_b37, position_b37, COUNT(*) as count FROM thegwensproject.genomiX.variants WHERE chr_id_b37 = '" +str(chr) +"' AND position_b37 = " + str(pos) + " GROUP BY chr_id_b37, position_b37 HAVING COUNT(*) > 1 LIMIT 20;")
    return query
query_job = client.query(count_variants(6, 166956723))
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row)

In [ ]:
def variant_info(chr, pos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.variants WHERE chr_id_b37 = '" +str(chr) +"' AND position_b37 = " + str(pos) + " LIMIT 20;")
    return query

query_job = client.query(variant_info(6, 166956723))
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row[9])
    print (row)

In [ ]:
def tag_disease_variant_info(chr, pos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.disease_variant_gene WHERE tag_chrom = '" +str(chr) +"' AND tag_pos = "+ str(pos) +"  LIMIT 20;")
    return query

query_job = client.query(tag_disease_variant_info(6, 166980468))
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row['lead_pos'])
    print (row)

In [75]:
def lead_disease_variant_info(chr, pos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.disease_variant_gene WHERE lead_chrom = '" +str(chr) +"' AND lead_pos = "+ str(pos) +"  LIMIT 20;")
    return query
query_job = client.query(lead_disease_variant_info(6, 166956723))
for row in query_job:
    # Row values can be accessed by field name or index.
    print (row)

In [ ]:
def exact_disease_variant_info(leadchr, leadpos, tagchr, tagpos):
    query = ( "SELECT * FROM thegwensproject.genomiX.disease_variant_gene WHERE lead_chrom = '" +str(leadchr) +"' AND lead_pos = "+ str(leadpos) +" AND tag_chrom = '" +str(tagchr) +"' AND tag_pos = "+ str(tagpos) +"  LIMIT 20;")
    return query
query_job = client.query(exact_disease_variant_info(6, 166980468, 6, 166952906))
for row in query_job:
    # Row values can be accessed by field name or index.
    print (row)
    print (row['trait_reported'])

In [ ]:
def lead_disease_info(chr, pos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.variant_disease WHERE lead_chrom = '" +str(chr) +"' AND lead_pos = "+ str(pos) +"  LIMIT 20;")
    return query
query_job = client.query(lead_disease_info(6, 166956723))
for row in query_job:
    # Row values can be accessed by field name or index.
    print (row['trait_reported'])

In [ ]:
def tag_disease_info(chr, pos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.variant_disease WHERE tag_chrom = '" +str(chr) +"' AND tag_pos = "+ str(pos) +"  LIMIT 20;")
    return query
query_job = client.query(tag_disease_info(6, 166952906))
for row in query_job:
    # Row values can be accessed by field name or index.
    print (row['trait_reported'])
    print (row['lead_chrom'] + " "+ str(row['lead_pos']))

In [ ]:
def exact_disease_info(leadchr, leadpos, tagchr, tagpos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.variant_disease WHERE tag_chrom = '" +str(tagchr) +"' AND tag_pos = "+ str(tagpos) +" AND lead_chrom = '" +str(leadchr) +"' AND lead_pos = "+ str(leadpos) +"  LIMIT 20;")
    return query
query_job = client.query(exact_disease_info(6, 166980468, 6, 166952906))
for row in query_job:
    # Row values can be accessed by field name or index.
    print (row['trait_reported'])

In [ ]:
def disease_catcher(disease):
    query = f'SELECT * FROM thegwensproject.genomiX.variant_disease WHERE trait_reported = "{disease}" LIMIT 20;'
    return query
query_job = client.query(disease_catcher("Alzheimer's disease"))
for row in query_job:
    # Row values can be accessed by field name or index.
    print (row['lead_chrom'] + " "+ str(row['lead_pos']))

In [ ]:
#use % of different parameters like acestry, for the disease specific genes.



In [ ]:
def lead_chrom_trait():
    query = f'SELECT lead_chrom, trait_reported, COUNT(*) AS count FROM thegwensproject.genomiX.disease_variant_gene GROUP BY lead_chrom, trait_reported;'
    return query
query_job = client.query(disease_catcher())
for row in query_job:
    # Row values can be accessed by field name or index.
    print (row['lead_chrom'] + " "+ str(row['lead_pos']))